In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import logging
# import warnings

# logging.basicConfig(level=logging.DEBUG)
# warnings.filterwarnings('ignore')

In [4]:
# import tensorflow as tf

# tf.compat.v1.enable_eager_execution()

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
config = malaya_speech.config.fastspeech2_config
config['encoder_hidden_size'] = 192
config['encoder_num_hidden_layers'] = 6
config['encoder_attention_head_size'] = 32
config['encoder_intermediate_size'] = 768
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [7]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [8]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [9]:
data['text_ids'][0].shape

(72,)

In [10]:
# i = tf.convert_to_tensor([data['text_ids'][0],data['text_ids'][0]])
# i_lengths = tf.convert_to_tensor([data['len_text_ids'][0,0], data['len_text_ids'][0,0]])
# mel_outputs = tf.convert_to_tensor([data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]])
# mel_lengths = tf.convert_to_tensor([408,408])

In [11]:
model = glowtts.Model(config, config_glowtts)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [12]:
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_) = model(i, y = mel_outputs, y_lengths = mel_lengths, training = True)
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Tensor("glowtts/StopGradient_5:0", shape=(?, ?, ?), dtype=float32) Tensor("glowtts/mul_2:0", shape=(?, 1, ?, ?), dtype=float32)


((<tf.Tensor 'glowtts/transpose_2:0' shape=(?, 80, ?) dtype=float32>,
  <tf.Tensor 'glowtts/transpose_10:0' shape=(?, 80, ?) dtype=float32>,
  <tf.Tensor 'glowtts/transpose_13:0' shape=(?, 80, ?) dtype=float32>,
  <tf.Tensor 'glowtts/FlowSpecDecoder/add_35:0' shape=<unknown> dtype=float32>,
  <tf.Tensor 'glowtts/transpose_1:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/transpose_5:0' shape=(?, 80, ?) dtype=float32>,
  <tf.Tensor 'glowtts/transpose_4:0' shape=(?, 80, ?) dtype=float32>,
  <tf.Tensor 'glowtts/transpose_3:0' shape=(?, 1, ?) dtype=float32>),
 (<tf.Tensor 'glowtts/ExpandDims_5:0' shape=(?, 1, ?, ?) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims_1:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'glowtts/mul_8:0' shape=(?, 1, ?) dtype=float32>))

In [13]:
# commons.mle_loss(z, z_m, z_logs, logdet, z_mask)

def mle_loss(z, m, logs, logdet, mask):
    l = tf.reduce_sum(logs) + 0.5 * tf.reduce_sum(tf.math.exp(-2 * logs) * ((z - m)**2))
    l = l - tf.reduce_sum(logdet)
    l = l / tf.reduce_sum(tf.ones_like(z) * mask)
    l = l + 0.5 * math.log(2 * math.pi)
    return l

l_mle = mle_loss(z, z_m, z_logs, logdet, tf.transpose(z_mask, [0, 2, 1]))
l_mle

<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [14]:
# commons.duration_loss(logw, logw_, x_lengths)

def duration_loss(logw, logw_, lengths):
    # l = torch.sum((logw - logw_)**2) / torch.sum(lengths)
    l = tf.reduce_sum((logw - logw_)**2) / tf.reduce_sum(tf.cast(lengths, tf.float32))
    return l

l_length = duration_loss(tf.transpose(logw, [0, 2, 1]), logw_, i_lengths)
l_length

<tf.Tensor 'truediv_1:0' shape=() dtype=float32>

In [15]:
loss_g = l_mle + l_length
loss_g

<tf.Tensor 'add_2:0' shape=() dtype=float32>

In [16]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 1e-4).minimize(loss_g)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [18]:
o_ = sess.run([l_mle, l_length], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[2.0197175, 4.1057777]

In [19]:
o_ = sess.run([attn, logw, logw_, x_mask], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[array([[[[1., 0., 0., ..., 0., 0., 0.],
          [0., 1., 0., ..., 0., 0., 0.],
          [0., 0., 1., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32),
 array([[[-0.5520879 ],
         [-1.3236997 ],
         [ 2.3119898 ],
         [ 5.0925727 ],
         [ 1.3901818 ],
         [ 0.11796892],
         [ 0.20959282],
         [ 0.85147285],
         [ 0.5525662 ],
         [-0.80431867],
         [ 1.4013798 ],
         [-1.7151494 ],
         [ 6.1689067 ],
         [ 1.3837274 ],
         [-0.7551726 ],
         [ 2.0265858 ],
         [ 1.0542395 ],
         [ 0.874967  ],
         [ 2.6840224 ],
         [ 0.46226084],
         [-1.2410879 ],
         [-0.9312018 ],
         [-0.04250371],
         [-2.9941368 ],
         [-3.7637963 ],
         [-1.4675146 ],
         [-0.88311017],
         [-3.1685643 ],
         [-1.0923625 ],
         [-2.4713845 ],


In [20]:
o_ = sess.run([z, z_m, z_logs, logdet, z_mask], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[array([[[ 2.338637 , -2.6748967,  2.5445697, ..., -2.27266  ,
           2.4386868, -2.27266  ],
         [-4.008886 ,  7.443169 , -3.742479 , ...,  7.5512137,
          -4.086241 ,  7.5512137],
         [ 2.9164453, -1.8286697,  3.0907717, ..., -2.27266  ,
           2.4386868, -2.27266  ],
         ...,
         [-2.2517886,  3.8982325, -2.1210325, ...,  7.5512137,
          -4.086241 ,  7.5512137],
         [ 1.467893 , -1.5104319,  1.4553391, ..., -2.27266  ,
           2.4386868, -2.27266  ],
         [-2.3140385,  4.1783476, -2.017513 , ...,  7.5512137,
          -4.086241 ,  7.5512137]]], dtype=float32),
 array([[[-1.065649  ,  1.612131  ,  1.612131  , ..., -0.7718891 ,
           0.7078509 ,  0.7078509 ],
         [-1.1602393 , -0.03929688, -0.03929688, ..., -1.2799424 ,
           0.7990326 ,  0.7990326 ],
         [-0.29416558, -1.6305162 , -1.6305162 , ..., -1.9579592 ,
          -1.2485372 , -1.2485372 ],
         ...,
         [ 0.4234789 ,  0.21945326,  0.21945326, ..., 

In [23]:
for k in range(100):
    print(k, sess.run([optimizer, l_mle, l_length], feed_dict = {i: data['text_ids'],
                                       i_lengths: data['len_text_ids'][:,0],
                            mel_outputs: data['mel'],
                            mel_lengths: data['len_mel'][:,0]}))

0 [None, 2.0082638, 3.9435637]
1 [None, 1.8867741, 2.915527]
2 [None, 1.7648637, 3.022738]
3 [None, 1.6627394, 3.3396132]
4 [None, 1.6562428, 3.1494272]
5 [None, 1.5459509, 3.1971414]
6 [None, 1.5924222, 2.9023976]
7 [None, 1.5135331, 2.2553358]
8 [None, 1.4745437, 4.25812]
9 [None, 1.4076732, 2.6180713]
10 [None, 1.3828757, 2.6491537]
11 [None, 1.3625342, 3.8854642]
12 [None, 1.3234448, 3.4123526]
13 [None, 1.310247, 4.03262]
14 [None, 1.2537138, 2.700779]
15 [None, 1.2240825, 3.2080307]
16 [None, 1.1867146, 2.5195649]
17 [None, 1.1406574, 2.9540052]
18 [None, 1.0746489, 2.7253084]
19 [None, 1.0614097, 2.853713]
20 [None, 0.984643, 2.2572656]
21 [None, 0.9768592, 1.9598135]
22 [None, 0.9409118, 2.3951418]
23 [None, 0.85064125, 2.1073885]
24 [None, 0.84987545, 3.1743176]
25 [None, 0.7958105, 1.8223108]
26 [None, 0.758583, 1.891879]
27 [None, 0.7238544, 1.8284637]
28 [None, 0.6632887, 1.3383814]
29 [None, 0.64890754, 2.2787218]
30 [None, 0.6107688, 1.358837]
31 [None, 0.5786559, 1.48190

In [24]:
tf.trainable_variables()

[<tf.Variable 'n_blocks_1/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_1/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_1/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_2/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_3/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_4/w_init:0' shape=(4, 4) dtype=float32_ref>,
 <tf.Variable 'n_blocks_5/logs:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf.Variable 'n_blocks_5/bias:0' shape=(1, 1, 160) dtype=float32_ref>,
 <tf

In [25]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [1]:
!ls -lh test
!rm -rf test

total 200792
-rw-r--r--  1 huseinzolkepli  staff    77B Oct  6 00:40 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    81M Oct  6 00:40 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    20K Oct  6 00:40 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    17M Oct  6 00:40 model.ckpt.meta
